In [1]:
# -*- coding: utf-8 -*-
"""mnist_dataset

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1ITUgSOqVvtEx9RaFMcoRVamk6B4J4CFV
"""

import os
import gzip
import numpy as np
import matplotlib.pyplot as plt 
import numpy as np
from tqdm import tqdm

class NeuralNetwork(object):
    def __init__(self, input_size, ouput_size):
        
        self.w = 0.2 * np.random.rand(input_size, ouput_size) - 0.1
        self.b = np.zeros(ouput_size)
        
    def forward(self, x):
        x = x.reshape(-1, 1) # add a new dimension to x
        output = self.w * x
        output = np.sum(output, axis=0)
        output = output + self.b.T
        return output
    
    def backward(self, output, label, image, learning_rate, epoch):
        #print(output.shape, label.shape, image.shape)
        output = softmax(output)
        error = label - output

        # Calculate the gradients
        grad_w = -np.outer(image, error.T)
        grad_b = -error.T

        # Update the weights and biases
        self.w -= learning_rate * grad_w
        self.b -= learning_rate * grad_b

  
def mean_squared_error(y, t):
    return 0.5 * np.sum((y - t)**2)

def softmax(a):
    sum_all = 0.0  # Initialize the sum
    for index in range(a.shape[0]):  # Iterate over the first dimension (rows)
        sum_all += np.exp(a[index])
    softmax_output = np.exp(a) / sum_all  # Apply softmax function
    return softmax_output

# dataset link : http://yann.lecun.com/exdb/mnist/
def read_images(file_path, normalize=True):
  with gzip.open(file_path,'rb') as f:
    features = np.frombuffer(f.read(), dtype=np.uint8, offset=16) 
    features =  features.reshape(-1, 784)
  
  if normalize:
    features = features.astype(np.float32)
    features = features / 255.0

  return features
    
def read_labels(file_path):
  with gzip.open(file_path, 'rb') as f:
    labels = np.frombuffer(f.read(), dtype=np.uint8, offset=8)

  return labels


def load_mnist(mnist_dir_path='./'):
  train_images_path = os.path.join(mnist_dir_path + 'train-images-idx3-ubyte.gz') 
  train_labels_path = os.path.join(mnist_dir_path + 'train-labels-idx1-ubyte.gz')
  test_images_path = os.path.join(mnist_dir_path + 't10k-images-idx3-ubyte.gz')
  test_labels_path = os.path.join(mnist_dir_path + 't10k-labels-idx1-ubyte.gz')

  train_images = read_images(train_images_path)
  train_labels = read_labels(train_labels_path)
  test_images = read_images(test_images_path)
  test_labels = read_labels(test_labels_path)

  return train_images, train_labels, test_images, test_labels

def show_image(image, label):
  plt.title("Label : {}".format(str(label)))
  plt.imshow(image.reshape(28, 28), cmap=plt.cm.gray_r)
  plt.show()

def one_hot_encoding(labels):
  output = np.zeros(shape=(labels.shape[0], 10))
  # zero matrix (size : (num of samples)x(num of class))
  for i in range(labels.shape[0]):
    output[i][labels[i]] = 1 
  
  return output

def train(model, train_images, train_labels):

    initial_learning_rate = 0.01
    decay_rate = 0.1
    decay_steps = 3

    learning_rate = initial_learning_rate
    epochs = 12
    
    valid_images, valid_labels = train_images[55000:], train_labels[55000:]
    train_images, train_labels = train_images[:55000], train_labels[:55000]
    
    
    valid_labels = one_hot_encoding(valid_labels)
    train_labels = one_hot_encoding(train_labels)
    
    for epoch in tqdm(range(epochs)):
        epoch_loss = []
        for idx, (image, label) in enumerate(zip(train_images, train_labels)):
                    
            output = model.forward(image) 
            soft_output = softmax(output)
            loss = mean_squared_error(soft_output, label)
            epoch_loss.append(loss)
            model.backward(output, label, image, learning_rate, epoch)
            
        print("train_loss : {}".format(np.mean(epoch_loss)))
        count = 0
        for image, label in zip(valid_images, valid_labels):
            
            output = model.forward(image) 
            
            predict_label = np.argmax(output)
            gold_label = np.argmax(label)
            
            if predict_label == gold_label:
              count += 1
        
        accuarcy = (count / len(valid_labels)) * 100 
        print("validation accuracy : {}".format(accuarcy))

        # Learning Rate Scheduling
        if ((epoch+1) % decay_steps == 0) and (epoch>=4):
           learning_rate *= decay_rate

    return model
    

def test(model, test_images, test_labels):
   
  count = 0
  for image, label in zip(test_images, test_labels):
      output = model.forward(image) 
      
      predict_label = np.argmax(output)
      #gold_label = np.argmax(label)
      
      
      if predict_label == label:
        count += 1
  
  accuracy = (count / len(test_labels)) * 100
  print("test accuracy : {}".format(accuracy))

In [2]:
if __name__ == "__main__":
  mnist_dir_path = '/Users/hyeokseung/경희대학교/2학년 1학기/인공지능개론/Perceptron/'
  train_images, train_labels, test_images, test_labels =  load_mnist(mnist_dir_path)

  #show_image(train_images[0], train_labels[0])

  
  model = NeuralNetwork(784, 10)
  
  model = train(model, train_images, train_labels)
  
  result = test(model, test_images, test_labels)
  

  8%|▊         | 1/12 [00:04<00:50,  4.57s/it]

train_loss : 0.07901779197407477
validation accuracy : 93.10000000000001


 17%|█▋        | 2/12 [00:09<00:45,  4.56s/it]

train_loss : 0.06494298740406186
validation accuracy : 93.38


 25%|██▌       | 3/12 [00:13<00:40,  4.56s/it]

...

 92%|█████████▏| 11/12 [00:50<00:04,  4.57s/it]

train_loss : 0.05413466836465241
validation accuracy : 94.22


100%|██████████| 12/12 [00:54<00:00,  4.56s/it]

train_loss : 0.054099660856634445
validation accuracy : 94.24


test accuracy : 92.52
